# Ray Tasks Revisited

© 2019-2022, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademyLogo.png)

The [Ray Crash Course](../ray-crash-course/00-Ray-Crash-Course-Overview.ipynb) introduced the core concepts of Ray's API and how they parallelize work. Specifically, we learned how to define Ray _tasks_ and _actors_, run them, and retrieve the results. 

This lesson explores Ray tasks in greater depth, including the following:

* How task dependencies are handled automatically by Ray
* Usage patterns for `ray.get()` and `ray.wait()`
* Specifying limits on the number of invocations and retries on failure
* An exploration of task granularity considerations
* Profiling tasks

In [1]:
import ray, time, os, sys 
import numpy as np 
sys.path.append("..")
from util.printing import pd, pnd  # convenience methods for printing results.

In [2]:
ray.init(ignore_reinit_error=True)

2022-03-15 15:42:02,151	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-03-15_15-41-59_972290_66714/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-15_15-41-59_972290_66714/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-03-15_15-41-59_972290_66714',
 'metrics_export_port': 64873,
 'gcs_address': '127.0.0.1:64530',
 'address': '127.0.0.1:64530',
 'node_id': '5b3fb103e23266d81ca2121e3a1da1afab6b7192884695348d794628'}

The Ray Dashboard URL is printed above and also part of the output dictionary item `webui_url`

(When using the Anyscale platform, use the URL provided by your instructor to access the Ray Dashboard.)

## Ray Task Dependencies

Let's define a few remote tasks, which will have _dependency_ relationships. We'll learn how Ray handles these dependent, asynchronous computations.

One task will return a random NumPy array of some size `n` and the other task will add two such arrays. We'll also add a sleep time, one tenth the size of `n` to simulate expensive computation.

> **Note:** Dependencies and how Ray implements handling of them are explored in depth in the [03: Ray Internals](03-Ray-Internals.ipynb) lesson. 

In [3]:
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

Now define a task that can add two NumPy arrays together. The arrays need to be the same size, but we'll ignore any checking for this requirement.

In [4]:
@ray.remote
def add_arrays(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

Now lets use them!

In [5]:
start = time.time()
ref1 = make_array.remote(20)
ref2 = make_array.remote(20)
ref3 = add_arrays.remote(ref1, ref2)
print(ray.get(ref3))
pd(time.time() - start, prefix="Total time:")

[-0.45910671  1.82919838  1.19195871  0.18379313 -0.04599046 -0.51902731
  1.61301122 -1.11785356  1.0189966  -0.98376257 -0.66115255  1.16808859
 -0.43705541 -1.28884547  0.45202732 -3.02796611  0.50733132 -0.35182225
 -0.84618178 -0.29480743]
Total time: duration:  4.142 seconds


Something subtle and "magical" happened here; when we called `add_arrays`, we didn't need to call `ray.get()` first for `ref1` and `ref2`, since `add_arrays` expects NumPy arrays. Because `add_arrays` is a Ray task, Ray automatically does the extraction for us, so we can write code that looks more natural and Pythonic.

Furthermore, note that the `add_arrays` task effectively depends on the outputs of the two `make_array` tasks. Ray won't run `add_arrays` until the other tasks are finished. Hence, Ray automatically handles task dependencies for us.

This is why the elapsed time is about 4 seconds. We used a size of 20, so we slept 2 seconds in each call to `make_array`, but those happened in parallel, _followed_ by a second sleep of 2 seconds in `add_arrays`.

Even though three task invocations occurred, we only used one call to `ray.get()`, when we actually needed the final results. Eliminating unnecessary `ray.get()` calls helps avoid forcing tasks to become synchronous when they could be asynchronous. So, keep these two key points in mind:

* _Don't ask for results you don't need._
* _Don't ask for the results you need until you really need them._

We don't need to see the objects for `id1` and `id2`. We only need the final array for `id3`.

## Using ray.wait() with ray.get()

Here is an idiomatic way to use `ray.get()`, where we fire all five asynchronous tasks at once, then ask for all the results at once with `ray.get()`:

In [6]:
start = time.time()

# Comprehension list: five NumPy object references or futures created
array_refs = [make_array.remote(n*10) for n in range(5)]

# Comprehension list: object references or futures of the result of addition
added_array_refs = [add_arrays.remote(ref, ref) for ref in array_refs]

# Iterate o er the list of object references or futures
for array in ray.get(added_array_refs):
    print(f'{array.size}: {array}')
pd(time.time() - start, prefix="Total time:")

0: []
10: [-0.34820369  2.83325277  0.98406322  1.11185361  4.63379185  0.56715043
  1.40371751  1.23905694 -0.12422939 -0.55029556]
20: [ 3.66427219  3.72671465  3.10813624  2.55002877  0.45014973  0.85337406
  2.51425252  1.24013323  0.56390205  1.38945543 -1.69721667  1.10418602
  1.40485479  0.90131341 -1.61866457  2.59150963  1.59432884 -4.1447867
  1.85227726 -2.48698443]
30: [ 0.23464612  2.36222213  0.94718626  0.33542855  1.38799444  1.64892408
 -5.39403202 -1.73740676  1.54192053 -1.86160749 -4.03107569 -2.70662954
 -2.04105437 -1.66803105 -1.44624183  1.69455724  1.70327388 -1.88295115
  0.41240878  2.98050304  3.78961575 -3.69821084 -2.57847957  1.40876894
 -0.66866115  5.64868333  0.2226802   0.96586096 -4.0921747  -1.37512701]
40: [ 0.85648582  0.97162374 -0.58435844 -1.27297238 -4.70113182  1.63973696
  1.06799225  1.92238403 -1.00852361  2.34573651  1.5775251  -3.74594951
 -2.20562833  2.42915957  1.5336613   1.10310627 -2.35991739 -1.85524639
 -2.14735621 -0.89891354 -

This takes about eight seconds, four seconds for the longest invocation invocation of `make_array`, `make_array(4)`  , and four seconds with longest invocation of `add_arrays`, when passed the results of `make_array(4)`. 

We did the right thing inside each list comprehension. We started the asynchronous tasks all at once and allowed Ray to handle the dependencies. Then we waited on one `ray.get()` call for all the output. 

However, what you see is no output and then everything is suddenly printed at once after eight seconds.

There are two fundamental problems with the way we've used `ray.get()` so far:

1. There's no timeout, in case something gets "hung".
2. We have to wait for _all_ the objects to be available before `ray.get()` returns.

The ability to specify a timeout is essential in production code as a defensive measure. Many potential problems could happen in a real production system, any one of which could cause the task we're waiting on to take an abnormally long time to complete or never complete. Our application would be deadlocked waiting on this task. Hence, it's **strongly recommended** in production software to always use timeouts on blocking calls, so that the application can attempt some sort of recovery in situations like this, or at least report the error and "degrade gracefully".

Actually, there _is_ a `timeout=<value>` option you can pass to `ray.get()` ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get)), but it will most likely be removed in a future release of Ray. Why remove it if timeouts are important? This change will simplify the implementation of `ray.get()` and encourage the use of `ray.wait()` for waiting ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.wait)) instead, followed by using `ray.get()` to retrieve values for tasks that `ray.wait()` tells us are finished. 

Using `ray.wait()` is also the way to fix the second problem with using `ray.get()` by itself, that we have to wait for all tasks to finish before we get any values back. Some of those tasks finish more quickly in our contrived example. We would like to process those results as soon as they are available, even while others continue to run. We'll use `ray.wait()` for this purpose.

Therefore, while `ray.get()` is simple and convenient, for _production code_, we recommend using `ray.wait()`, **with** timeouts, for blocking on running tasks. Then use `ray.get()` to retrieve values of completed tasks. 

Here is the previous example rewritten to use `ray.wait()`:

In [7]:
start = time.time()
array_refs = [make_array.remote(n*10) for n in range(5)]
added_array_refs = [add_arrays.remote(ref, ref) for ref in array_refs]

arrays = []
waiting_refs = list(added_array_refs)  # Assign a working list to the full list of refs
while len(waiting_refs) > 0:           # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of refs we're still waiting to complete,
    #   2. tell it to return immediately as soon as one of them completes,
    #   3. tell it wait up to 10 seconds before timing out.
    ready_refs, remaining_refs = ray.wait(waiting_refs, num_returns=1, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_refs), time.time() - start))
    new_arrays = ray.get(ready_refs)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_refs = remaining_refs  # Reset this list; don't include the completed refs in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   1 completed tasks. (elapsed time:  0.003)
0: []
Returned   1 completed tasks. (elapsed time:  2.009)
10: [ 2.61249698  1.15353876 -0.19799867  0.17900869 -1.25084314  3.01916945
 -0.52149485  2.98897652 -0.15137511 -0.50667582]
Returned   1 completed tasks. (elapsed time:  4.011)
20: [-2.77511803 -0.70786392 -2.09859848 -0.18133085  2.09138509 -0.9301783
  1.91350371  0.25086912 -0.81192482 -3.41332779  1.85861711 -1.33658321
 -0.87961558 -1.85539659  2.62335247 -2.89981383  0.81628865 -2.20359524
 -3.36886916 -2.79150556]
Returned   1 completed tasks. (elapsed time:  6.010)
30: [-1.79365208  0.41225367 -0.39937561  1.80659174 -0.98909957  2.60732657
  1.51565835  0.53625763 -2.10054078  1.94146458 -4.00973662  4.00877276
 -0.48103527  0.36419716 -0.90711228 -0.77277181 -0.6674774  -0.55134704
 -1.28280809  0.23543636  0.13575372 -1.64472359 -5.37592876  0.1611564
 -2.20451254  4.44301538  0.7015706  -1.03992621 -1.85270783  1.04956613]
Returned   1 completed tasks. (elapsed

Now it still takes about 8 seconds to complete, 4 seconds for the longest invocation of `make_array` and 4 seconds for the invocation of `add_arrays`, but since the others complete more quickly, we see their results as soon as they become available, at 0, 2, 4, and 6 second intervals.

> **Warning:** For each call to `ray.wait()` in a loop like this, it's important to remove the refs that have completed. Otherwise, `ray.wait()` will return immediately with the same list containg the first completed item, over and over again; you'll loop forever!! Resetting the list is easy, since the second list returned by `ray.wait()` is the rest of the items that are still running. So, that's what we use.

Now let's try it with `num_returns = 2`:

In [8]:
start = time.time()
array_refs = [make_array.remote(n*10) for n in range(5)]
added_array_refs = [add_arrays.remote(ref, ref) for ref in array_refs]

arrays = []
waiting_refs = list(added_array_refs)  # Assign a working list to the full list of refs
while len(waiting_refs) > 0:           # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of refs we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_refs) > 1 else 1
    ready_refs, remaining_refs = ray.wait(waiting_refs, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_refs), time.time() - start))
    new_arrays = ray.get(ready_refs)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_refs = remaining_refs  # Reset this list; don't include the completed refs in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.011)
0: []
10: [ 2.85010557  0.52080099 -0.17394622  0.90507833 -1.92617361  0.57061709
  2.20247151  1.63692052 -0.64660581 -0.46026788]
Returned   2 completed tasks. (elapsed time:  6.008)
20: [ 0.2687956   1.64642862  2.89730022 -3.05491581  2.4487754  -0.09083297
 -2.38509223  1.41448073  0.77264342 -1.39549426  2.15904105  3.18070418
  1.62280584  0.31176733  1.5453099  -0.98422432 -0.68148331  0.24257604
 -2.54901168  0.43292972]
30: [-1.82089699 -0.46458805 -0.92392108 -1.8241863  -0.14463016  0.89706231
  1.45097526  0.06274639 -1.95920819 -0.79733678  3.65431647  0.61554247
 -2.19547177 -0.55598758  1.91643679  3.44241103  1.13479906 -1.77712453
 -1.27724718 -3.25624224  1.62819833 -0.41085036  2.07553874 -2.72892165
  0.69555361 -0.87795738 -3.3923477   1.17418179 -2.52775984  2.72694429]
Returned   1 completed tasks. (elapsed time:  8.011)
40: [-8.75029345e-01 -2.22969699e-01 -1.52254323e+00 -1.78851663e-01
 -1.10892734e+00 -1.

Now we get two at a time output. Note that we don't actually pass `num_returns=2` every time. If you ask for more items than the length of the input list, you get an error. So, we compute `num_returns`, using `2` except when there's only one task to wait on, in which case we use `1`. So, in fact, the output for `40` was a single task result, because we started with `5` and processed two at a time.

 For a longer discussion on `ray.wait()`, see [this blog post](https://medium.com/distributed-computing-with-ray/ray-tips-and-tricks-part-i-ray-wait-9ed7a0b9836d).

## Exercise 1

The following cell is identical to the last one. Modify it to use a timeout of `2.5` seconds, shorter than our longest tasks. What happens now? Try using other times.

See the [solutions notebook](solutions/Advanced-Ray-Solutions.ipynb) for a discussion of this exercise and the subsequent exercises.

In [9]:
start = time.time()
array_refs = [make_array.remote(n*10) for n in range(5)]
added_array_refs = [add_arrays.remote(ref, ref) for ref in array_refs]

arrays = []
waiting_refs = list(added_array_refs)  # Assign a working list to the full list of refs
while len(waiting_refs) > 0:           # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of refs we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_refs) > 1 else 1
    ready_refs, remaining_refs = ray.wait(waiting_refs, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_refs), time.time() - start))
    new_arrays = ray.get(ready_refs)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_refs = remaining_refs  # Reset this list; don't include the completed refs in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.014)
0: []
10: [-3.96764478  0.59125109 -1.43949307  0.94717244 -1.06301074 -2.73518899
 -1.94045808 -0.7318014  -1.43338887 -0.01363469]
Returned   2 completed tasks. (elapsed time:  6.013)
20: [ 0.92782943 -1.24323041 -0.09998991  1.79173307  1.98688311 -1.38655334
 -0.12801522  2.0554255   2.66364926 -0.2381073  -6.07929461  0.10890244
 -1.56105774  0.53199683 -4.44274757  2.43556271 -2.79581128 -0.29052248
 -4.2318059   0.40789263]
30: [ 3.13224157 -0.36451071 -0.29795524 -1.75725106  0.84876122  4.34212685
  0.43521279  3.52888278 -2.52134082 -2.11022208  0.174927    2.14232855
 -0.7075055  -0.13135602 -2.92231604  1.01338399  0.61940899  0.32957764
  2.05709252  2.87469281 -2.05782654  0.59831843 -1.35034379 -1.19082869
  3.03349211 -0.33284876 -4.49177625 -3.47023573 -0.96549628  0.42274481]
Returned   1 completed tasks. (elapsed time:  8.013)
40: [ 3.19377317 -2.3343181  -0.815824   -2.23217324 -2.13800149  0.33605759
  1.44256358

In conclusion:

> **Tips:**
>
> 1. Use `ray.wait()` with a timeout to wait for one or more running tasks. Then use `ray.get()` to retrieve the values for the finished tasks.
> 2. When looping over calls to `ray.wait()` with a list of object refs for running tasks, remove the previously-completed and retrieved objects from the list.
> 3. Don't ask for results you don't need.
> 4. Don't ask for the results you need until you really need them.

## Exercise 3

Let's practice converting a slow loop to Ray, including using `ray.wait()`. Change the function to be a Ray task. Change the invocations to use the `ray.wait()` idiom. You can just use the default values for `num_returns` and `timeout` if you want. The second cell uses `assert` statements to check your work.

In [ ]:
def slow_square(n):
    time.sleep(n)
    return n*n

start = time.time()
squares = [slow_square(n) for n in range(4)]
for square in squares:
    print (f'finished: {square}')
duration = time.time() - start

In [ ]:
assert squares == [0, 1, 4, 9], f'Did you use ray.get() to retrieve the values? squares = {squares}'
assert duration < 4.1, f'Did you use Ray to parallelize the work? duration = {duration}' 

## Limiting Task Invocations and Retries on Failure

> **Note:** This feature may change in a future version of Ray. See the latest details in the [Ray documentation](https://docs.ray.io/en/latest/package-ref.html#ray.remote). 

Two options you can pass to `ray.remote` when defining a task affect how often it can be invoked and retrying on failure:

* `max_calls`: This specifies the maximum number of times that a given worker can execute the given remote function before it must exit. This can be used to address memory leaks in third-party libraries or to reclaim resources that cannot easily be released, e.g., GPU memory that was acquired by TensorFlow. By default this is infinite.
* `max_retries`: This specifies the maximum number of times that the remote function should be rerun when the worker process executing it crashes unexpectedly. The minimum valid value is 0, the default is 4, and a value of -1 indicates infinite retries are allowed.

Example:

```python
@ray.remote(max_calls=10000, max_retries=10)
def foo():
    pass
```

See the [ray.remote()](https://docs.ray.io/en/latest/package-ref.html#ray.remote) documentation for all the keyword arguments supported.

### Overriding with config()

Remote task and actor objects returned by `@ray.remote` can also be dynamically modified with the same arguments supported by `ray.remote()` using `options()` as in the following examples:

```python
@ray.remote(num_gpus=1, max_calls=1, num_return_vals=2)
def f():
    return 1, 2
g = f.options(num_gpus=2, max_calls=None)
```

## What Is the Optimal Task Granularity

How fine-grained should Ray tasks be? There's no fixed rule of thumb, but Ray clearly adds some overhead for task management and using object stores in a cluster. Therefore, it makes sense that tasks which are too small will perform poorly.

We'll explore this topic over several more lessons, but for now, let's get a sense of the overhead while running in your setup.

We'll continue to use NumPy arrays to create "load", but remove the `sleep` calls:

In [10]:
def noop(n):
    return n

def local_make_array(n):
    return np.random.standard_normal(n)

@ray.remote
def remote_make_array(n):
    return local_make_array(n)

Let's do `trials` runs for each experiment, to average out background noise:

In [11]:
trials=100

First, let's use `noop` to baseline local function calls. Note that we call `print` for the duration, rathern than `pd`, because the overhead is so low the `pd` formatting will print `0.000`:

In [12]:
start = time.time()
[noop(t) for t in range(trials)]
print(f'{time.time() - start} seconds')

0.00012683868408203125 seconds


Let's try the same run with `local_make_array(n)` for `n = 100000`:

In [13]:
start = time.time()
[local_make_array(100000) for _ in range(trials)]
print(f'{time.time() - start} seconds')

0.27930593490600586 seconds


So, we can safely ignore the "noop" overhead for now. For completeness, here's what happens with remote execution:

In [14]:
start = time.time()
refs = [remote_make_array.remote(100000) for _ in range(trials)]
ray.get(refs)
print(f'{time.time() - start} seconds')

0.1549530029296875 seconds


For arrays of 100000, using Ray is faster (at least on this test machine). The benefits of parallel computation, rather than synchronous, already outweight the Ray overhead.

## Exercise 4

1. Try incrementing size of n to 2n
2. Do you see a marked difference between the local vs remote times?
3. For the brave, try using `mathplotlib` to plot it.

## Profiling Tasks with ray.timeline()

Sometimes you need to debug performance problems in Ray tasks. Calling `ray.timeline(file)` ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.timeline)) captures profiling information for subsequent task execution to the specified file. Afterwards, you can view the data in a Chrome web browser. The format used is unique to Chrome, so Chrome has be used to view the data.

Let's try it with our `make_array` and `add_arrays` methods in the following code. First some potential cleanup:

In [16]:
timeline_file = 'task-timeline.txt' # Will be found in the same directory as this notebook.
if os.path.isfile(timeline_file):   # Delete old one, if an old one exists already.
    os.remove(timeline_file)

In [17]:
ray.timeline(timeline_file)
start = time.time()
array_refs = [make_array.remote(n*10) for n in range(5)]
added_array_refs = [add_arrays.remote(ref, ref) for ref in array_refs]
for array in ray.get(added_array_refs):
    print(f'{array.size}: {array}')
pd(time.time() - start, prefix="Total time:")

0: []
10: [ 0.44588725 -0.30504069  1.43863312  2.32652261  0.06712068 -3.04394395
  0.54379724 -3.0151979  -0.97603632  0.45520804]
20: [-1.58438362  0.55177065 -0.36981937  2.00179322  0.62362295  2.31395488
  0.97234654 -3.05845891  1.62513001  1.50521922 -2.56475263 -0.53216708
  1.22913291  0.22820228 -2.09324559 -4.30758317  0.45049657  1.6692868
 -1.29869988 -2.07318162]
30: [-2.15344363  0.59048439 -0.57666621 -1.45349224 -2.21326358 -0.35332541
 -1.1042098   3.84878396  1.10634569  1.28813825 -1.84321214  3.24516481
 -1.41934703 -1.05559117  2.06570698  0.79661935  0.3252803  -0.7247974
 -2.59285391  4.98506072  1.99052716  1.36111418  0.02545001 -1.82408576
 -2.05127259  1.80831505 -1.86420724  0.22428651  3.54284682  0.12737484]
40: [-0.18757363  0.4513238  -1.28031893 -0.69429297  0.06664299  0.12053353
 -2.88507391 -0.32675906  4.47810585 -1.26819809  2.67983322 -0.64027334
  1.14544518 -1.48665216  1.48000624  3.04515058  1.44912493 -0.62272859
  2.87327188 -2.0450877   2

Now, to view the data:

1. Open Chrome and enter chrome://tracing.
2. Click the _Load_ button to load the `task-timeline.txt` file, which will be in this notebook's directory. 
3. To zoom in or out, click the "asymmetric" up-down arrow button. Then hold the mouse button in the graph and roll the mouse scroll wheel up or down. (On a laptop trackpad, press and hold, then use another finger to slide up and down.)
4. To move around, click the crossed arrow and drag a section in view. 
5. Click on a box in the timeline to see details about it. 

Look for blocks corresponding to long-running tasks and look for idle periods, which reflect processing outside the context of Ray.

Here is a screen grab profiling the previous code, zoomed in on one block of tasks and with one task selected. Note the processes shown on the left for drivers (more than one notebook was running at this time) and workers.

![Ray Trace Example](../images/Ray-Trace-Example.png)

In [18]:
ray.shutdown()  # "Undo ray.init()".

The next lesson, [Ray Actors Revisited](02-Ray-Actors-Revisited.ipynb), revisits actors. It provides a more in-depth look at actor characteristics and profiling actor performance using the _Ray Dashboard_.